# Testing

In [2]:
!conda install pytest --yes # this package is not included in Scipy Notebook

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytest


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    atomicwrites-1.3.0         |             py_0           9 KB  conda-forge
    importlib_metadata-0.18    |           py37_0          36 KB  conda-forge
    more-itertools-4.3.0       |        py37_1000          83 KB  conda-forge
    pluggy-0.12.0              |             py_0          18 KB  conda-forge
    py-1.8.0                   |             py_0          65 KB  conda-forge
    pytest-5.0.0               |           py37_0         348 KB  conda-forge
    zipp-0.5.1                 |             py_0           7 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         566 KB

The following NEW packages will be INSTALLED:

  atomicw

In [3]:
import pytest